In [1]:
import pandas as pd
import numpy as np
import json
import os
from datasets import Dataset
from transformers import *
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import re
import gc
import emoji
from tqdm.notebook import tqdm
print('TF version',tf.__version__)


TF version 2.6.2


In [2]:
def strip_emoji(text):
    new_text = re.sub(emoji.get_emoji_regexp(), r"", text)
    return new_text
def remove_links(text):
    return re.sub(r'http\S+', '', text)

In [3]:
with open("../input/twitter-bot-detection/Twibot-20/Twibot-20/train.json") as f:
    train_json = json.load(f)
    train_df = pd.json_normalize(train_json, sep = '_')

In [4]:
train_df = train_df.explode('tweet')

In [5]:
# train_df.domain.apply(len).max() #not useful don't have in ukraine

In [6]:
train_df.head()

,ID,tweet,neighbor,domain,label,profile_id,profile_id_str,profile_name,profile_screen_name,profile_location,...,profile_profile_link_color,profile_profile_sidebar_border_color,profile_profile_sidebar_fill_color,profile_profile_text_color,profile_profile_use_background_image,profile_has_extended_profile,profile_default_profile,profile_default_profile_image,neighbor_following,neighbor_follower
0,17461978,RT @CarnivalCruise: 🎉 Are you ready to see wha...,NaN,"[Politics, Business, Entertainment]",0,17461978,17461978,SHAQ,SHAQ,"Orlando, FL",...,2FC2EF,181A1E,252429,666666,True,False,False,False,NaN,NaN
0,17461978,Who has time for receipts? Not me. @epson rece...,NaN,"[Politics, Business, Entertainment]",0,17461978,17461978,SHAQ,SHAQ,"Orlando, FL",...,2FC2EF,181A1E,252429,666666,True,False,False,False,NaN,NaN
0,17461978,Steady wants to encourage you to invest in you...,NaN,"[Politics, Business, Entertainment]",0,17461978,17461978,SHAQ,SHAQ,"Orlando, FL",...,2FC2EF,181A1E,252429,666666,True,False,False,False,NaN,NaN
0,17461978,"Good one, @rishid. But let’s see if y'all can ...",NaN,"[Politics, Business, Entertainment]",0,17461978,17461978,SHAQ,SHAQ,"Orlando, FL",...,2FC2EF,181A1E,252429,666666,True,False,False,False,NaN,NaN
0,17461978,#lsunationalchamps\n,NaN,"[Politics, Business, Entertainment]",0,17461978,17461978,SHAQ,SHAQ,"Orlando, FL",...,2FC2EF,181A1E,252429,666666,True,False,False,False,NaN,NaN


In [7]:
# sample = train_df.sample(500).reset_index(drop = True)
meta = ['profile_name', 'profile_description', 'profile_followers_count']
train_df = train_df[meta + ['tweet', 'label']]
train_df = train_df.dropna().sample(100)
X = train_df[['tweet', 'label']]
X_meta = train_df[meta]

In [8]:
X.columns = ['texts', 'labels']
X.labels = X.labels.apply(int)
X.texts = X.texts.apply(strip_emoji)
X.texts = X.texts.apply(remove_links)

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  


In [9]:
def compute_metrics(pred):
    labels = pred.label_ids
    #map to 01
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    roc = roc_auc_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [10]:
model_ckpt = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_ckpt)
#tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
#model = TFDistilBertModel.from_pretrained("distilbert-base-uncased")

https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp728u1jea


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
creating metadata file for /root/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpvj6e8at9


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmphipef_0d


Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
creating metadata file for /root/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
creating metadata file for /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.0

In [11]:
input_ids=[]
attention_masks=[]
tweets = X["texts"].to_numpy()
for tweet in tqdm(tweets):
    bert_inp=tokenizer.encode_plus(tweet,add_special_tokens = False, max_length =128, pad_to_max_length = True,return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
target = np.array(pd.get_dummies(X['labels']))

  0%|          | 0/100 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [12]:
input_ids[0]

array([ 1030,  2630, 15006,  2102,  2034,  2091,  2059,  2039,  2085,
        2091,  2153,  1029,  2023,  2003,  6135, 21873,   999,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

In [13]:
# del X, train_df
# gc.collect()

In [14]:
AUTOTUNE = tf.data.AUTOTUNE

# Note that some tokenizers also returns 'token_id'. Modify this function accordingly. 
@tf.function
def parse_data(from_tokenizer, target):
    input_ids = from_tokenizer['input_ids']
    attention_mask = from_tokenizer['attention_mask']
    
    target = tf.cast(target, tf.float32)
    
    return {'input_ids': input_ids,
            'attention_mask': attention_mask}, target

In [15]:
# this should be faster, but its not :-S
# train = dict()
# train['input_ids'] = train_tokens
# train['attention_mask'] = train_attention
# trainloader = tf.data.Dataset.from_tensor_slices((train, y_train.reshape(-1, 1)))
# #testloader = tf.data.Dataset.from_tensor_slices((test_tokens, list(val_label)))
# trainloader = (
#         trainloader
#         .shuffle(32)
#         .map(parse_data, num_parallel_calls=AUTOTUNE)
#         .batch(batch_size)
#         .prefetch(AUTOTUNE)
#     )

# # validloader = (
# #         validloader
# #         .map(parse_data, num_parallel_calls=AUTOTUNE)
# #         .batch(batch_size)
# #         .prefetch(AUTOTUNE)
# #     )

In [16]:
batch_size =32
# trainloader, validloader = get_dataloaders(train_encodings, y_train, val_encodings, y_test)

In [17]:
config = AutoConfig.from_pretrained(model_ckpt) 
backbone = TFDistilBertModel.from_pretrained(model_ckpt, config = config)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.16.2",
  "vocab_size": 30522
}

https://huggingface.co/distilbert-base-uncased/resolve/main/tf_model.h5 not found in cache or force_download set to True, downloading to /root/.cache/huggingface/tr

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/tf_model.h5 in cache at /root/.cache/huggingface/transformers/fa107dc22c014df078a1b75235144a927f7e9764916222711f239b7ee6092ec9.bc4b731be56d8422e12b1d5bfa86fbd81d18d2770da1f5ac4f33640a17b7dde9.h5
creating metadata file for /root/.cache/huggingface/transformers/fa107dc22c014df078a1b75235144a927f7e9764916222711f239b7ee6092ec9.bc4b731be56d8422e12b1d5bfa86fbd81d18d2770da1f5ac4f33640a17b7dde9.h5
loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/tf_model.h5 from cache at /root/.cache/huggingface/transformers/fa107dc22c014df078a1b75235144a927f7e9764916222711f239b7ee6092ec9.bc4b731be56d8422e12b1d5bfa86fbd81d18d2770da1f5ac4f33640a17b7dde9.h5
2022-03-29 09:10:01.700732: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Some layers from the model checkpoint at distilbert-base-uncased were not used whe

In [18]:
backbone.summary()

Model: "tf_distil_bert_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  66362880  
Total params: 66,362,880
Trainable params: 66,362,880
Non-trainable params: 0
_________________________________________________________________


In [19]:
def build_model():
    
    input_ids = tf.keras.Input(shape=(128,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(128,),dtype='int32')
    meta = tf.keras.Input(shape=(1,),dtype='int32') #just followers for now, don't know how to deal with rest
    
    output = backbone(input_ids,attention_masks)[0]
    
    output = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(5, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(output)
    output = tf.keras.layers.GlobalMaxPool1D()(output)
    
    output = tf.keras.layers.Dense(8,activation='relu')(output)
    output = tf.keras.layers.Dropout(0.2)(output)
    meta_output = tf.keras.layers.Dense(8,activation='relu')(meta)
    concat = tf.keras.layers.concatenate([meta_output, output], name = "concat")
    
    output = tf.keras.layers.Dense(2,activation='softmax', name = "head")(concat)
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks, meta],outputs = output)
    
    
    for layer in model.layers[:3]:
        layer.trainable = False
    
    return model

In [20]:
keras_model = build_model()

In [21]:
followers = X_meta[meta[2]].to_numpy().astype('float32')
X_train, X_test, y_train, y_test,train_mask,test_mask, meta_train, meta_test =train_test_split(input_ids,
                                                                       target,
                                                                       attention_masks,
                                                                       followers, test_size=0.3)

In [22]:
scaler = StandardScaler()
meta_train = scaler.fit_transform(meta_train.reshape(-1, 1))
meta_test = scaler.transform(meta_test.reshape(-1,1))

In [23]:
keras_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy', tf.keras.metrics.AUC()])

In [24]:
#leakage in metadata, to be fixed
history=keras_model.fit([X_train,train_mask, meta_train],y_train,batch_size=32,epochs=1,validation_split = 0.2)

2/2 [==============================] - 24s 7s/step - loss: 0.6869 - accuracy: 0.4821 - auc: 0.5202 - val_loss: 0.5923 - val_accuracy: 0.6429 - val_auc: 0.7092


In [25]:
tf.keras.models.save_model(
    keras_model,
    'withmeta',
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None,
    save_traces=True
)